# Imviz profiling

Create three channels of smoothly varying images:

In [ ]:
%%time
from time import sleep

import numpy as np
from scipy.ndimage import gaussian_filter
from astropy.nddata import NDData
from astropy.wcs import WCS

from jdaviz import Imviz

# pixel dimensions of the RGB image
shape = (600, 600)

# one side of the pixel, in degrees
pixel_width = 0.005

# offset the centers of the fields of view
# for the three images by this fraction of 
# one image's width:
center_offset = 0.2
delta_ra_offset = (
    center_offset * pixel_width * shape[0]
)

# the imviz viewport has a configurable size
imviz_height = 800  # [pix]

# compress images generated in the python kernel
# with this method before sending to the client:
# Choices: {'png' (default), 'webp', 'none'}
image_compression = 'png' 

# number of images to generate: 
n_images = 6


wcs_list = [
    WCS({
    'CTYPE1': 'RA---TAN', 
    'CUNIT1': 'deg', 
    'CDELT1': pixel_width,
    'CRPIX1': shape[0] / 2, 
    'CRVAL1': 180 + i * delta_ra_offset,
    'CTYPE2': 'DEC--TAN', 
    'CUNIT2': 'deg', 
    'CDELT2': pixel_width,
    'CRPIX2': shape[1] / 2, 
    'CRVAL2': 0
    }) for i in range(n_images)
]

random_shape = (6, 6)
image_list = [np.random.uniform(size=random_shape) for i in range(n_images)]

colors = ['c', 'y', 'm', 'r', 'g', 'b', 'o']

def low_pass_filter(x):
    rfft = np.fft.rfft2(x)
    irfft = np.fft.irfft2(rfft, shape)
    return 2**16 * irfft / irfft.max()

nddata_cym = [
    NDData(low_pass_filter(image), wcs=wcs) 
    for image, wcs in zip(image_list, wcs_list)
]

Initialize and show Imviz:

In [ ]:
%%time

imviz = Imviz()
imviz.app.layout.border = '2px solid rgb(143, 56, 3)'
imviz.app.get_viewer('imviz-0')._composite_image.compression = image_compression

label_mouseover = imviz.app.session.application._tools['g-coords-info']
viewer = imviz.default_viewer._obj

# imviz.show('sidecar:split-right', height=800)
imviz.show(height=imviz_height)

Load the images into Imviz:

In [ ]:
%%time

with imviz.batch_load():
    for nddata, data_label in zip(nddata_cym, colors):
        imviz.load_data(nddata, data_label=data_label)

compression = imviz.app.get_viewer('imviz-0')._composite_image.compression
print(f"glue-jupyter compression configuration: '{compression}'")

Use WCS to align the images, zoom to fit:

In [ ]:
%%time

orientation = imviz.plugins['Orientation']
orientation.align_by = 'WCS'

Rotate the image orientation by 180 deg counter-clockwise:

In [ ]:
%%time

# # This seems not to work, fails on call that's effectively `viewer.zoom(0)`. Bug?
# rotate the image by 180 deg (CCW):
# orientation.add_orientation(rotation_angle=180)

Assign colors to each of the layers:

In [ ]:
%%time

plot_options = imviz.plugins["Plot Options"]
for layer, color in zip(plot_options.layer.choices, colors):
    plot_options.layer = layer
    plot_options.image_color_mode = 'Color'
    plot_options.image_color = color
    plot_options.image_opacity = 0.7

Return to the default orientation:

In [ ]:
%%time

# # This seems not to work, fails on call that's effectively `viewer.zoom(0)`. Bug?
# orientation.orientation = 'Default orientation'

Blink through each layer (one visible at a time):

In [ ]:
%%time

for i in range(len(plot_options.layer.choices)):
    viewer.blink_once()

Make all layers visible again:

In [ ]:
%%time

for layer in plot_options.layer.choices:
    plot_options.layer = layer
    plot_options.image_visible = True

Increment through colormap upper limits for each layer:

In [ ]:
%%time

for vmax in np.linspace(0, 2**16, 10):
    for layer in plot_options.layer.choices:
        plot_options.layer = layer
        plot_options.stretch_vmin = 0
        plot_options.stretch_vmax = vmax